# Bayesian Networks and HMM

$
\newcommand{\x}{\mathbf{x}}
\newcommand{\y}{\mathbf{y}}
$

| | |
|--|--|
| **Names** | *waheed ud din siddiqui & Sanne van Waveren* |
| **Group** | *AC ML_09* |


In this lab, you are given a basic implementation of the components of Factor graphs and the sum-product algorithm. With this, you can compute marginal probabilities and, by incorporating extra factors, joint probabilities of nodes and observations. Familiarise yourself with the code, and then answer the questions below.


In [4]:
%matplotlib inline
import numpy as np
import scipy.stats as st
import scipy.ndimage as ndim
import matplotlib.pyplot as plt
import operator
import pylab

pylab.rcParams['figure.figsize'] = (10.0, 8.0)



In [5]:
class Factor:
    """Implement a factor node in a Bayesian Network"""
    def __init__(self,name,factor):
        """Initialisation with the name of the node, and the implemented function of the connected variables
        
        The function `factor` must take a tuple as argument, with as many elements in the tuple as there 
        are connected variables to the factor node and the elements of the argument being the indices of the 
        possible values in the variable node. It returns the factor value for that setting of variables.
        
        Example: a factor implementing p(A), with A being a boolean variable with possible values {0: True, 1: False},
        factor((1,)) should return p(~a).
        """
        self.name = name
        self.fn = factor
        self.connections = [] # Pointer to the nodes this factor is connected to
        self.msgRcvd = []     # Stores messages received from the connected nodes
        self.msgSent = []     # Has a message been sent to the connected nodes
        self.nMsgsRcvd = 0    # number of messages received
    
    def addConnection(self, node):
        """Register a connection to a variable. The connections must be added to the factor node
        in the same order as the indices are expected by the factor function
        
        This automatically registers the inverse connection from variable to the factor."""
        self.connections.append(node)
        self.msgSent.append(False)
        self.msgRcvd.append(None)
        node.addConnection(self)
        
    def __str__(self):
        return self.name
    def __repr__(self):
        return "Factor "+self.name
        
    def recPrint(self, parent=None, indent=""):
        """Recursively print the tree"""
        print indent,self.__str__()
        for i in range(len(self.connections)):
            if self.connections[i]!=parent:
                self.connections[i].recPrint(self,indent+"  ")
            
    def recvMsg(self, sender, msg):
        """Receive a message `msg` from a connected `sender` variable node"""
        print self.name,"Receiving message from",sender,":",msg
        for i in range(len(self.connections)):
            if self.connections[i] == sender:
                if self.msgRcvd[i]!=None:
                    print "... ignoring"
                    return
                self.msgRcvd[i] = msg
                self.nMsgsRcvd += 1
                return
        print "ERROR:",self.__str__(),"could not find the corresponding variable",sender.__str__()
   
    def resetMsgs(self):
        self.nMsgsRcvd=0
        self.msgRcvd = [ None for m in self.msgRcvd ]
        self.msgSent = [ False for m in self.msgSent ]

    def recursiveEnumerate(self, outIndex, valIndices, values, res):
        """Stores, in list `res`, for each possible value of the node connected to connection `outindex`,
            the sum over all possible combinations of values of the incoming connections of the product 
            of the factor of these values with the messages with these values. (Aaaargh.) """
        if len(valIndices) == len(self.connections): # end of recursion
            prod = self.fn(tuple(valIndices))
            for i in range(len(valIndices)):
                if i != outIndex:
                    prod *= self.msgRcvd[i][valIndices[i]]
            
            res[valIndices[outIndex]] += prod 
            return
        
        for i,v in enumerate(self.connections[len(valIndices)].values):
            self.recursiveEnumerate(outIndex,valIndices+[i],values+[v],res)
        
        
    def sendMessages(self):
        """Send a message to all connections on which no message has been sent yet, if all
        other connections have received a message already"""        
        
        # Special case: single connection
        if not self.msgSent[0] and len(self.connections) == 1:
            self.msgSent[0] = True    # Record that a message has been sent on that connection
            self.connections[0].recvMsg(self,np.array([self.fn((i,)) for i,v in enumerate(self.connections[0].values) ]))
            return True
        
        if self.nMsgsRcvd+1<len(self.connections): # More than one connection without incoming message
            return False                           # Can't send anything
        
        # If one connection has not received it's message, send a msg out on that connection
        if self.nMsgsRcvd+1 == len(self.connections): 
            for i in range(len(self.connections)):    # For connections on which we could send a message                
                if self.msgRcvd[i] == None:           # Find the one that hasn't received a message yet
                    if not self.msgSent[i]:           # If no message has been sent on that connection yet
                        msg = np.array([ 0. for c in self.connections[i].values ]) # Create a message with an entry for each possible value of the outgoing node
                        self.recursiveEnumerate(i,[],[],msg) # Compute the message value
                        self.connections[i].recvMsg(self,msg) 
                        self.msgSent[i] = True
                        return True
            
        if self.nMsgsRcvd == len(self.connections): # message received on all connections, send wherever not done yet.
            sentAny = False
            for i in range(len(self.connections)):       # For connections on which we could send a message
                if self.msgSent[i]:                      # ignore if handled already
                    continue
                msg = np.array([ 0. for c in self.connections[i].values ])

                self.recursiveEnumerate(i,[],[],msg)
                self.connections[i].recvMsg(self,msg)
                self.msgSent[i] = True
                sentAny = True
            return sentAny
                            
        return False
            
    

In [12]:
class Variable:
    """Implement a Variable node with categorical values"""
    def __init__(self,name,values):
        """Initialise with the name of the variable and a list of possible values for that variable"""
        self.name = name
        self.connections = []
        self.values = values
        self.msgRcvd = []     # Stores messages received from the connected nodes
        self.msgSent = []     # Has a message been sent to the connected nodes
        self.nMsgsRcvd = 0    # number of messages received
    
    def addConnection(self, node):
        """Add a connection to a factor node. This should never be called explicitly, except by
        the addConnection method of the Factor class"""
        self.connections.append(node)
        self.msgSent.append(False)
        self.msgRcvd.append(None)
        
    def __str__(self):
        return self.name
    def __repr__(self):
        return "Var "+self.name
    
    
    def recPrint(self, parent=None, indent=""):
        """Recursively print the tree"""
        print indent,self.__str__()
        for i in range(len(self.connections)):
            if self.connections[i]!=parent:
                self.connections[i].recPrint(self,indent+"  ")        
    
    def recvMsg(self, sender, msg):
        """Receive a message `msg` from Factor node `sender`"""
        print self.name,"Receiving message from",sender,":",msg
        for i in range(len(self.connections)):
            if self.connections[i] == sender:
                if self.msgRcvd[i]!=None:
                    print "WARNING: ignoring a message"
                    return
                self.msgRcvd[i] = msg
                self.nMsgsRcvd += 1
                return
        print "ERROR:",self.__str__(),"could not find the corresponding factor",sender.__str__()
        
    def resetMsgs(self):
        self.nMsgsRcvd=0
        self.msgRcvd = [ None for m in self.msgRcvd ]
        self.msgSent = [ False for m in self.msgSent ]
        
    
    def sendMessages(self):
        """Send a message to all connections on which no message has been sent yet, if all
        other connections have received a message already
        """    
        # Special case: single connection
        if not self.msgSent[0] and len(self.connections) == 1:
            self.msgSent[0] = True
            self.connections[0].recvMsg(self,np.array([1. for v in self.values]))
            return True
        
        # If one connection has not received it's message, send a msg out on that connection
        if self.nMsgsRcvd+1 == len(self.connections): 
            con = 0
            for i in range(len(self.connections)):    # For connections on which we could send a message                
                if self.msgRcvd[i] == None:           # Find the one that hasn't received a message yet
                    con = i
                    break
                    
            if self.msgSent[con]:                     # Don't send it again
                return False
            
            msg = np.ones(len(self.values))           # The message is the product of the 
            for i in range(len(self.connections)):    # incoming messages on all connections,
                if i == con:                          # except the outgoing one
                    continue
                msg *= self.msgRcvd[i]
            self.connections[con].recvMsg(self,msg)   # Send the message
            self.msgSent[con] = True
            return True
            
        if self.nMsgsRcvd == len(self.connections): # message received on all connections, send wherever not done yet.
            sentAny = False
            for con in range(len(self.connections)):       # For connections on which we could send a message
                if self.msgSent[con]:                      # ignore if handled already
                    continue
                msg = np.ones(len(self.values))            # Initialise the message
                for i in range(len(self.connections)):     # multiply with all relevant incoming messages
                    if i == con:
                        continue
                    msg *= self.msgRcvd[i]
                self.connections[con].recvMsg(self,msg)    # and send it out
                self.msgSent[con] = True                
                sentAny = True
            return sentAny

        return False
    
    def getMarginals(self):
        """Once all messages have been received, the marginal probability 
        (or joint probability with specific values of other nodes, if such 
        factors have been implemented), is the product of all incoming messages"""
        if self.nMsgsRcvd != len(self.connections):        # Check that all messages have been received
            print "ERROR printing marginals: messages missing"
        prod = self.msgRcvd[0].copy()# Compute the product
        for m in self.msgRcvd[1:]:
            prod *= m
        print "gamma fair coin: ", prod[1] / (prod[0] + prod[1])
        return prod                              

In [13]:
class Graph:
    def __init__(self):
        self.var = {}
        self.fact = {}
    def addVariables(self,values, *names):
        for n in names:
            self.var[n] = Variable(n,values)
    def addFactor(self,name,fTable, *connections):
        self.fact[name] = Factor(name,lambda(v) : fTable[v])
        for c in connections:
            self.fact[name].addConnection(self.var[c])
    def propagateMessages(self):
        hasSent = True
        while hasSent:
            hasSent = False
            for k,f in self.fact.iteritems():
                hasSent |= f.sendMessages()
            for k,v in self.var.iteritems():
                hasSent |= v.sendMessages()
    def printMarginals(self):
        # Print out the marginals
        for k,v in self.var.iteritems():
            print v,v.getMarginals()
        
    def reset(self):
        for k,f in self.fact.iteritems():
            f.resetMsgs()
        for k,v in self.var.iteritems():
            v.resetMsgs()
        

pB = np.array([1./12,11./12])
pC = np.array([1./3,2./3])
pA_BC = np.array([[[0, 0.25],[.5,1.]],[[1,.75],[.5,0.]]])
pD_A = np.array([[0,.75],[1,.25]])

g = Graph()
g.addVariables(["True","False"], "A","B","C","D")
g.addFactor("p(B)", pB, "B")
g.addFactor("p(C)", pC, "C")
g.addFactor("p(A|B,C)", pA_BC, "A","B","C")
g.addFactor("P(D|A)",pD_A,"D","A")

g.propagateMessages()
print "Marginal probabilities:"
g.printMarginals()
g.reset()

#print "\n"
#g.addFactor("obs B", np.array([1.,0.]),"B")
#g.propagateMessages()
#g.printMarginals()


B Receiving message from p(B) : [ 0.08333333  0.91666667]
C Receiving message from p(C) : [ 0.33333333  0.66666667]
p(A|B,C) Receiving message from C : [ 0.33333333  0.66666667]
p(A|B,C) Receiving message from B : [ 0.08333333  0.91666667]
P(D|A) Receiving message from D : [ 1.  1.]
A Receiving message from p(A|B,C) : [ 0.77777778  0.22222222]
A Receiving message from P(D|A) : [ 1.  1.]
p(A|B,C) Receiving message from A : [ 1.  1.]
P(D|A) Receiving message from A : [ 0.77777778  0.22222222]
B Receiving message from p(A|B,C) : [ 1.  1.]
C Receiving message from p(A|B,C) : [ 1.  1.]
D Receiving message from P(D|A) : [ 0.16666667  0.83333333]
p(C) Receiving message from C : [ 1.  1.]
p(B) Receiving message from B : [ 1.  1.]
Marginal probabilities:
A gamma fair coin:  0.222222222222
[ 0.77777778  0.22222222]
C gamma fair coin:  0.666666666667
[ 0.33333333  0.66666667]
B gamma fair coin:  0.916666666667
[ 0.08333333  0.91666667]
D gamma fair coin:  0.833333333333
[ 0.16666667  0.83333333]


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:65: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:93: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


** Question 1 [10 marks]** Using the above code, implement the computation of the following probabilities:

* $p(B|\neg A)$
* $p(C|\neg A)$
* $p(C|\neg A,\neg B)$
* $p(\neg A, \neg B, C, \neg D)$
* $p(B,C|D)$

In [5]:
# Answer Q2

# BAYES RULE: 
# P(B | not A) = P(not A | B)  * P (B) / P(not A) = (0.06944444*0.08333333) / 0.22222222 =            0.02604

# P(C | not A) = P (not A | C) * P(C) / P(not A) = (0.18055556*0.33333333) / 0.22222222  =            0.27083

# P(C | not A, not B) = P(C, notA, notB) / P(notA,notB) = 0.15277778 / 0.15277778  =                  1

# P(notA, notB, C, notD) =                                                                            0.03819444

# P (B, C | D) = P(B, C, D) / P(D) = 0.02083333 / 0.16666667 =                                        0.12450

#~~~~~~~~~~~~~~~~~~~~~
#print "\n"
#print "Case 1:"
#g.addFactor("obs B", np.array([1.,0.]),"B")
#g.propagateMessages()
#g.printMarginals()
#g.reset()
#~~~~~~~~~~~~~~~~~~~~~
#print "\n"
#print "Case 2:"
#g.addFactor("obs C", np.array([1.,0.]),"C")
#g.propagateMessages()
#g.printMarginals()
#g.reset()
#~~~~~~~~~~~~~~~~~~~~~
#print "\n"
#print "Case 3a:"
#g.addFactor("obs C", np.array([1.,0.]),"C")
#g.addFactor("obs not B", np.array([0.,1.]),"B")
#g.addFactor("obs not A", np.array([0.,1.]),"A")
#g.propagateMessages()
#g.printMarginals()
#g.reset()
#print "Case 3b:"
#g.addFactor("obs not B", np.array([0.,1.]),"B")
#g.addFactor("obs not A", np.array([0.,1.]),"A")
#g.propagateMessages()
#g.printMarginals()
#g.reset()
#~~~~~~~~~~~~~~~~~~~~~
#print "\n"
#print "Case 4:"
#g.addFactor("obs C", np.array([1.,0.]),"C")
#g.addFactor("obs not B", np.array([0.,1.]),"B")
#g.addFactor("obs not D", np.array([0.,1.]),"D")
#g.addFactor("obs not A", np.array([0.,1.]),"A")
#g.propagateMessages()
#g.printMarginals()
#g.reset()
#~~~~~~~~~~~~~~~~~~~~~
#print "\n"
#print "Case 5:"
#g.addFactor("obs C", np.array([1.,0.]),"C")
#g.addFactor("obs B", np.array([1.,0.]),"B")
#g.addFactor("obs D", np.array([1.,0.]),"D")
#g.propagateMessages()
#g.printMarginals()
#g.reset()

#print "\n"
#print "Test case P(A|B) = 1/6:"
#g.addFactor("obs B", np.array([1.,0.]),"B")
#g.propagateMessages()
#g.printMarginals()
#g.reset()




Test case P(A|B) = 1/6:
A [ 0.01388889  0.06944444]
C [ 0.02777778  0.05555556]
B [ 0.08333333  0.        ]
D [ 0.05208333  0.03125   ]


c:\python27\lib\site-packages\ipykernel\__main__.py:65: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
c:\python27\lib\site-packages\ipykernel\__main__.py:93: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.



Somebody is cheating at playing heads and tails: he's using a biased coin (probability of heads: 0.7). However, to avoid suspicion, he also uses a fair coin (probability of heads: 0.5) sometimes, and switches between the two coins occasionally. If he has just thrown a fair coin, he will switch to the biased coin with probability 0.2 (and keep the fair coin with probability 0.8). If he's using the biased coin, he'll switch to fair with probability 0.5. When he starts playing, he chooses a coin at random (p=0.5)

You're playing with this person and observe the following sequence of outcomes:

$$
HHTHHHTTTHTH
$$

**Question 2 [20 marks]** Using the above code, implement a graph corresponding to the above problem. This is an example of a Hidden Markov Model. Given the probabilities listed above, what is the probability of this sequence? For each throw, what is the probability that a fair coin was used? 


In [192]:
# Answer Q2

# probabilities, initial, transition and emission
prior = np.array([0.5,0.5])
pH_Z = np.array([0.7, 0.5])
pT_Z = np.array([0.3, 0.5])
transprobs = np.array([[0.5, 0.5], [0.2, 0.8]])

# h factor, described by (13.45) in the course book.
h = prior * pH_Z

f2 = transprobs * pH_Z
f3 = transprobs * pT_Z
f4 = f2
f5 = f2
f6 = f2
f7 = f3
f8 = f3
f9 = f3
f10 = f2
f11 = f3
f12 = f2
f13 = np.array([1,1])

g = Graph()
g.addVariables(["Bias", "Fair"], "Z1", "Z2", "Z3", "Z4", "Z5", "Z6", "Z7", "Z8", "Z9", "Z10", "Z11", "Z12")

#factor h(Z), representing P(Z1|S), it has a connection with Z1 (S = start of observation)
g.addFactor("P(Z1|S)", h, "Z1")
#factor f1, representing P(Z2|Z1), it has a connection with Z1 and Z2.
g.addFactor("P(Z2|Z1)", f2, "Z1", "Z2")
g.addFactor("P(Z3|Z2)", f3, "Z2", "Z3")
g.addFactor("P(Z4|Z3)", f4, "Z3", "Z4")
g.addFactor("P(Z5|Z4)", f5, "Z4", "Z5")
g.addFactor("P(Z6|Z5)", f6, "Z5", "Z6")
g.addFactor("P(Z7|Z6)", f7, "Z6", "Z7")
g.addFactor("P(Z8|Z7)", f8, "Z7", "Z8")
g.addFactor("P(Z9|Z8)", f9, "Z8", "Z9")
g.addFactor("P(Z10|Z9)", f10, "Z9", "Z10")
g.addFactor("P(Z11|Z10)", f11, "Z10", "Z11")
g.addFactor("P(Z12|Z11)", f12, "Z11", "Z12")
g.addFactor("P(Z12|end)", f13, "Z12")
g.propagateMessages()
g.printMarginals()
g.reset()

# For the probability of the sequence given this model we marginalize over Z4.
# Adding Z4[0] and Z4[1] gives:
# P(O|M) = 0.00010107  + 0.00020507 = 0.00030614

# We want the posterior distribution p(fair coin|x1, . . . , xN) of zn given the observed data set x1, . . . , xN.
# This is the alpha*beta for the fair coin, divided by the sum of alpha*beta values for both coins (see page 620 of the course book)
# Thus, the probability for each throw to be a fair coin:
# Throw 1: 0.399217439891
# Throw 2: 0.568913199651
# Throw 3: 0.739388667524
# Throw 4: 0.631022251712
# Throw 5: 0.610952844646
# Throw 6: 0.651943393877
# Throw 7: 0.809623930505
# Throw 8: 0.839144619275
# Throw 9: 0.820867335175
# Throw 10: 0.705037249262
# Throw 11: 0.781295292667
# Throw 12: 0.669864215216

Z12 Receiving message from P(Z12|end) : [1 1]
Z1 Receiving message from P(Z1|S) : [ 0.35  0.25]
Z11 Receiving message from P(Z12|Z11) : [ 0.6   0.54]
Z2 Receiving message from P(Z2|Z1) : [ 0.1575  0.1875]
Z3 Receiving message from P(Z3|Z2) : [ 0.034875  0.114375]
Z10 Receiving message from P(Z11|Z10) : [ 0.225  0.252]
Z4 Receiving message from P(Z4|Z3) : [ 0.02821875  0.05446875]
Z9 Receiving message from P(Z10|Z9) : [ 0.14175  0.1323 ]
Z5 Receiving message from P(Z5|Z4) : [ 0.01750219  0.02884219]
Z8 Receiving message from P(Z9|Z8) : [ 0.0543375  0.061425 ]
Z7 Receiving message from P(Z8|Z7) : [ 0.02350687  0.02783025]
Z6 Receiving message from P(Z6|Z5) : [ 0.01016367  0.01591242]
Z6 Receiving message from P(Z7|Z6) : [ 0.01048359  0.01254251]
Z7 Receiving message from P(Z7|Z6) : [ 0.0024793   0.00890589]
Z8 Receiving message from P(Z8|Z7) : [ 0.00090625  0.00418218]
Z5 Receiving message from P(Z6|Z5) : [ 0.00680489  0.00648471]
Z4 Receiving message from P(Z5|Z4) : [ 0.00400289  0.0035

c:\python27\lib\site-packages\ipykernel\__main__.py:65: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
c:\python27\lib\site-packages\ipykernel\__main__.py:93: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [193]:
# Control with the homework assignment values, to check if we are correct.

# probabilities, initial, transition and emission
prior = np.array([0.6,0.4])
pH_Z = np.array([0.2, 0.6])
pT_Z = np.array([0.8, 0.4])
transprobs = np.array([[0.3, 0.7], [0.6, 0.4]])

# h factor, described by (13.45) in the course book.
h = prior * pH_Z

f2 = transprobs * pT_Z
f3 = f2
f4 = transprobs * pH_Z
f5 = np.array([1, 1])


g = Graph()
g.addVariables(["Fair", "Bias"], "Z1", "Z2", "Z3", "Z4")
#,"Z4", "Z5", "Z6", "Z7", "Z8", "Z9", "Z10", "Z11", "Z12"

#factor h(Z), representing P(Z1|S), it has a connection with Z1 (S = start of observation)
g.addFactor("P(Z1|S)", h, "Z1")
#factor f1, representing P(Z2|Z1), it has a connection with Z1 and Z2.
g.addFactor("P(Z2|Z1)", f2, "Z1", "Z2")
g.addFactor("P(Z3|Z2)", f3, "Z2", "Z3")
g.addFactor("P(Z4|Z3)", f4, "Z3", "Z4")
g.addFactor("P(Z4|end)", f5, "Z4")

g.propagateMessages()
g.printMarginals()
g.reset()

# For the probability of the sequence given this model add Z4[0] and Z4[1], which gives:
# 0.010368 + 0.041472 = 0.05184 

Z4 Receiving message from P(Z4|end) : [1 1]
Z1 Receiving message from P(Z1|S) : [ 0.12  0.24]
Z3 Receiving message from P(Z4|Z3) : [ 0.48  0.36]
Z2 Receiving message from P(Z2|Z1) : [ 0.144  0.072]
Z2 Receiving message from P(Z3|Z2) : [ 0.216  0.288]
Z3 Receiving message from P(Z3|Z2) : [ 0.06912  0.05184]
Z4 Receiving message from P(Z4|Z3) : [ 0.010368  0.041472]
Z1 Receiving message from P(Z2|Z1) : [ 0.13248  0.14976]
Z4 gamma fair coin:  0.8
[ 0.010368  0.041472]
Z1 gamma fair coin:  0.693333333333
[ 0.0158976  0.0359424]
Z2 gamma fair coin:  0.4
[ 0.031104  0.020736]
Z3 gamma fair coin:  0.36
[ 0.0331776  0.0186624]


c:\python27\lib\site-packages\ipykernel\__main__.py:65: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
c:\python27\lib\site-packages\ipykernel\__main__.py:93: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [194]:
#Check 2

# Answer Q2

# probabilities, initial, transition and emission
prior = np.array([0.6,0.4])
pH_Z = np.array([0.2, 0.6])
pT_Z = np.array([0.8, 0.4])
transprobs = np.array([[0.3, 0.7], [0.6, 0.4]])

# h factor, described by (13.45) in the course book.
h = prior * pT_Z

f2 = transprobs * pH_Z
f3 = transprobs * pT_Z
f4 = f3
f5 = f2
f6 = f2
f7 = f3
f8 = np.array([1,1])
#pA_BC = np.array([[[0, 0.25],[.5,1.]],[[1,.75],[.5,0.]]])
#pD_A = np.array([[0,.75],[1,.25]])

g = Graph()
g.addVariables(["Fair", "Bias"], "Z1", "Z2", "Z3", "Z4", "Z5", "Z6", "Z7")

#factor h(Z), representing P(Z1|S), it has a connection with Z1 (S = start of observation)
g.addFactor("P(Z1|S)", h, "Z1")
#factor f1, representing P(Z2|Z1), it has a connection with Z1 and Z2.
g.addFactor("P(Z2|Z1)", f2, "Z1", "Z2")
g.addFactor("P(Z3|Z2)", f3, "Z2", "Z3")
g.addFactor("P(Z4|Z3)", f4, "Z3", "Z4")
g.addFactor("P(Z5|Z4)", f5, "Z4", "Z5")
g.addFactor("P(Z6|Z5)", f6, "Z5", "Z6")
g.addFactor("P(Z7|Z6)", f7, "Z6", "Z7")
g.addFactor("P(Z7|end)", f8, "Z7")
g.propagateMessages()
g.printMarginals()
g.reset()

# For the probability of the sequence given this model add Z4[0] and Z4[1], which gives:
# 0.00067369  + 0.00040033 = 0.00107402

Z7 Receiving message from P(Z7|end) : [1 1]
Z1 Receiving message from P(Z1|S) : [ 0.48  0.16]
Z6 Receiving message from P(Z7|Z6) : [ 0.52  0.64]
Z2 Receiving message from P(Z2|Z1) : [ 0.048  0.24 ]
Z3 Receiving message from P(Z3|Z2) : [ 0.12672  0.05184]
Z5 Receiving message from P(Z6|Z5) : [ 0.3    0.216]
Z4 Receiving message from P(Z5|Z4) : [ 0.10872  0.08784]
Z4 Receiving message from P(Z4|Z3) : [ 0.055296  0.043776]
Z5 Receiving message from P(Z5|Z4) : [ 0.00857088  0.03373056]
Z3 Receiving message from P(Z4|Z3) : [ 0.050688  0.06624 ]
Z2 Receiving message from P(Z3|Z2) : [ 0.03071232  0.03492864]
Z6 Receiving message from P(Z6|Z5) : [ 0.00456192  0.0116951 ]
Z7 Receiving message from P(Z7|Z6) : [ 0.00670851  0.00314855]
Z1 Receiving message from P(Z2|Z1) : [ 0.01651277  0.01206835]
Z4 gamma fair coin:  0.390104341972
[ 0.00601178  0.00384528]
Z5 gamma fair coin:  0.739145069
[ 0.00257126  0.0072858 ]
Z6 gamma fair coin:  0.759340289465
[ 0.0023722   0.00748487]
Z7 gamma fair coin:

c:\python27\lib\site-packages\ipykernel\__main__.py:65: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
c:\python27\lib\site-packages\ipykernel\__main__.py:93: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
